In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
#load data
train=pd.read_csv("/kaggle/input/titanic/train.csv")
train.head()
train.tail() #quick look at first and last few rows

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.00,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.00,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.45,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q


In [3]:
test=pd.read_csv("/kaggle/input/titanic/test.csv")
test.head()
test.tail() #quick look at first and last few rows

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S
417,1309,3,"Peter, Master. Michael J",male,NaN,1,1,2668,22.3583,NaN,C


In [4]:
women=train.loc[train.Sex=='female']["Survived"]
rateW=sum(women)/len(women)
print("Surviving members of the fairer sex: ", rateW)

men=train.loc[train.Sex=='male']["Survived"]
rateM=sum(men)/len(men)
print("Surviving members of the other sex: ", rateM ," :o")

Surviving members of the fairer sex:  0.7420382165605095
Surviving members of the other sex:  0.18890814558058924  :o


In [5]:
from sklearn.ensemble import RandomForestClassifier

#establish features for random forest predictor
y=train["Survived"] #desired output for prediction
feats=["Pclass", "Sex", "Age" ,"SibSp", "Parch"] #TODO: deal w/ Nan cols
feats2=["Survived", "Pclass", "Sex", "Age" ,"SibSp", "Parch"]

#indicator vals for relevant cols, converted from categorical
X=pd.get_dummies(train[feats])
X_test=pd.get_dummies(test[feats])

#remove Nan vals
X=pd.DataFrame(X)
pd.isna(X)
X=X.dropna()
X_test=pd.DataFrame(X_test)
X_test=X_test.dropna()

#problem: now y is the wrong size. How to fix?
#soln: same procedure as above
train2=pd.DataFrame(train[feats2])
train2=train2.dropna()
train2

y2=train2["Survived"]
y2

0      0
1      1
2      1
3      1
4      0
      ..
885    0
886    0
887    1
889    1
890    0
Name: Survived, Length: 714, dtype: int64

In [6]:
X_test

,Pclass,Age,SibSp,Parch,Sex_female,Sex_male
0,3,34.5,0,0,False,True
1,3,47.0,1,0,True,False
2,2,62.0,0,0,False,True
3,3,27.0,0,0,False,True
4,3,22.0,1,1,True,False
...,...,...,...,...,...,...
409,3,3.0,1,1,True,False
411,1,37.0,1,0,True,False
412,3,28.0,0,0,True,False
414,1,39.0,0,0,True,False


In [7]:
#create predictor and run on test data
model=RandomForestClassifier(n_estimators=50, max_depth=5, random_state=1)
model.fit(X, y2)
preds=model.predict(X_test)

preds.size #332
test.PassengerId #418

0       892
1       893
2       894
3       895
4       896
       ... 
413    1305
414    1306
415    1307
416    1308
417    1309
Name: PassengerId, Length: 418, dtype: int64

In [8]:
#submit. displaf
out=pd.DataFrame({"PassengerId": test.PassengerId, "Survived": preds})
out.to_csv('submission.csv', index=False)
print("BOOM")

ValueError: array length 332 does not match index length 418